In [2]:
from fastai.vision.all import *
from fastai.vision.widgets import *
import urllib.request
import re

# <center> The Plant Classifier </center>

# I. Classify with URL link

In [3]:
path = Path()
learn_inf = load_learner(path/'model.pkl', cpu=True)

In [4]:
def classify(url):
    urllib.request.urlretrieve(url,"image.jpg")
    img = PILImage.create("image.jpg")

    pred,pred_idx,probs = learn_inf.predict(img)
    return pred, probs[pred_idx].tolist(), probs

#Test
# link = "https://www.dropbox.com/s/gx7aq2eyrt0l5s5/Agrostemma-githago_Foliage_Substrat3_28022020%20%2873%29.JPG?raw=1"
# answer = classify(link)
# print(answer)


In [30]:
def classify_on_click(btn):
    htmlResult.value = "<h3>                                Analysing </h3>"
    display_image(txtURL.value)
    pred_class,probility,_=classify(txtURL.value)
    tree_type = re.search(r'.+?(?=_)',pred_class).group()
    growth_stage = re.search(r'(?<=_)[\w+.-]+',pred_class).group()
    html = "<p style='font-family:Arial;font-size:110%;text-align:center'>Plant type: <b>" + tree_type + "</b><br>" + "Growth stage: <b>" + growth_stage + "</b><br>" + "Probability: <b>" + "{:1.2f}".format(probility*100) + "</b> %</p>"
    htmlResult.value = html

# # Test
# ex = 'Crepis-setosa_Foliage'
# tree_type = re.search(r'.+?(?=_)',ex).group()
# growth_stage = re.search(r'(?<=_)[\w+.-]+',ex).group()
# tree_type, growth_stage
# pred_class,probility,_=classify(txtURL.value)
# pred_class,probility

In [31]:
def display_image(url):
    htmlImage.value= "<img src='" + url + "'width='500'>"


In [32]:
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.HTML()

In [33]:
def example_on_click(btn):
  txtURL.value = btn.tag
  classify_on_click(btnClassify)


In [34]:
def create_example(url):
  img = widgets.HTML("<img src='" + url + "' height='300'>")
  btn = widgets.Button(description="Test")
  btn.tag = url
  btn.on_click(example_on_click)
  vbox = widgets.VBox([img,btn])
  vbox.layout.align_items = "center"
  return vbox

In [35]:
def on_data_change(change):
    lbl_pred.value = ''
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(500,500))
    pred,pred_idx,probs = learn_inf.predict(img)
    tree_type = re.search(r'.+?(?=_)',pred).group()
    growth_stage = re.search(r'(?<=_)[\w+.-]+',pred).group()
    lbl_pred.value = "<p style='font-family:Arial;font-size:110%;'>Plant type: <b>" + tree_type+ "</b><br>" "Growth stage: <b>" + growth_stage + "</b><br>" + "Probability: <b>" + f'{probs[pred_idx]*100:1.2f}' +"</b> %</p>" 

In [36]:
btn_upload.observe(on_data_change, names=['data'])

In [41]:
DEFAULT_IMG = "https://www.dropbox.com/s/gx7aq2eyrt0l5s5/Agrostemma-githago_Foliage_Substrat3_28022020%20%2873%29.JPG?raw=1"

htmlImage = widgets.HTML(value="")
display_image(DEFAULT_IMG)

txtURL = widgets.Text(description="URL:", placeholder=" Enter the URL of a sugarbeet plant picture")
txtURL.layout.width = "500px"
btnClassify = widgets.Button(description="Classify")
btnClassify.on_click(classify_on_click)
boxH = widgets.HBox([txtURL, btnClassify])

htmlResult = widgets.HTML(value="")
# htmlResult.layout.margin = "0px 0px 0px 0px"
HBox([htmlResult])


boxV = widgets.VBox([htmlImage, boxH, htmlResult])
boxV.layout.align_items = "center"
display(boxV)

### Testing samples:

In [38]:
urls = [
  "https://www.dropbox.com/s/wrl6ehdivktfz3p/Agrostemma-githago_Cotyledon_Substrat3_12022020%20%28229%29.JPG?raw=1", 
  "https://www.dropbox.com/s/1t3enu6vjbxr1zr/Agrostemma-githago_Cotyledon_Substrat3_12022020%20%28258%29.JPG?raw=1",
  "https://www.dropbox.com/s/gx7aq2eyrt0l5s5/Agrostemma-githago_Foliage_Substrat3_28022020%20%2873%29.JPG?raw=1",
  "https://www.dropbox.com/s/8rd35rn1aj5mxe1/Agrostemma-githago_Foliage_Substrat3_28022020%20%28216%29.JPG?raw=1", 
  "https://www.dropbox.com/s/h8256ag2hwssdui/Agrostemma-githago_Intermediate_Substrat3_19022020%20%2829%29.JPG?raw=1",
  "https://www.dropbox.com/s/zh5ps93g1xdl5po/Agrostemma-githago_Intermediate_Substrat3_19022020%20%2863%29.JPG?raw=1",
  "https://www.dropbox.com/s/1ilv2qazazsk9q3/Beta-vulgaris_Cotyledon_Substrat1_07102019%20%289%29.JPG?raw=1", 
  "https://www.dropbox.com/s/slx8ss6a0ogbdix/Beta-vulgaris_Intermediate_Substrat1_01092019%20%2851%29.JPG?raw=1", 
  "https://www.dropbox.com/s/c3gmm8b08v821qu/Crepis-setosa_Foliage_Substrat1_17022020%20%281%29.JPG?raw=1"
  ]

items = []
for url in urls:
    items.append(create_example(url))

grid = widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(3,33% )"))
display(grid)

GridBox(children=(VBox(children=(HTML(value="<img src='https://www.dropbox.com/s/wrl6ehdivktfz3p/Agrostemma-gi…

# II. Classify by uploading images from your computer

In [14]:

my_box = VBox([widgets.Label('Select your plant!'), out_pl, btn_upload,  lbl_pred])
my_box.layout.align_items = "center"
display(my_box)